## Basic Algorithms Lab: Fibonacci and Insertion Sort


Another form of recursion is tree recursion. Consider computing a fibonacci sequence, in which each number is the sum of the previous two, with the first two taken to be 0 and 1.

### Recursive Fibonacci. 


We write Fibonacci recursively with the first two numbers as base cases. (as in the last lab)

![](https://mitpress.mit.edu/sicp/full-text/book/ch1-Z-G-13.gif)

(from SICP)

Signature: `def fib_recursive(n)`

In [4]:
def fib_recursive(n):
    if n == 1:
        return 0
    if n == 2:
        return 1
    return fib_recursive(n-1) + fib_recursive(n-2)

for i in [1, 2, 7, 13, 29, 33]:
    print(fib_recursive(i))

0
1
8
144
317811
2178309


In [11]:
fib_recursive(7)

8

### Q1. 

What are the space and time complexities of this implementation? Is this implementation tail-recursive?

*your answer here*

Time complexity is exponential. Space complexity is also exponential. The implementation is not tail-recursive. 

### Dynamic Programming and Iteration

From Skiena
>..**dynamic programming**, which typically removes one element from the problem, solves the smaller problem, and then uses the solution to this smaller problem to add back the element in the proper way. **Divide-and-conquer** instead splits the problem in (say) halves, solves each half, then stitches the pieces back together to form a full solution.

>Dynamic programming is a technique for efficiently implementing a recursive algorithm by storing partial results. The trick is seeing whether the naive recursive algorithm computes the same subproblems over and over and over again. If so, storing the answer for each subproblems in a table to look up instead of recompute can lead to an efficient algorithm. Start with a recursive algorithm or definition. Only once we have a correct recursive algorithm do we worry about speeding it up by using a results matrix. Dynamic programming is generally the right method for optimization problems on combinatorial objects that have an inherent left to right order among components. nents. Left-to-right objects includes: character strings, rooted trees, polygons, and integer sequences.

### Q2.

Here is an implementation of Fibonacci using dynamic programming: they key is to notice that the recurrence we used can be put into an iterative form and just stored in an everr increasing array. What is the space and time complexity here?

In [5]:
def fib_iterative(n):
    fibs=[]
    fibs.append(0)
    fibs.append(1)
    for i in range(2, n):
         fibs.append(fibs[i-1]+fibs[i-2])
    return fibs[n-1]

In [9]:
for i in [1, 2, 7,13,29,33]:
    print(fib_iterative(i))

0
1
8
144
317811
2178309


*your answer here*

Time complexity is linear. Space complexity is also linear. 

### Recursive Fibonacci with cacheing

In [114]:
def cache(f):
    """a single argument function whose values may be cached"""
    cache = {}
    def memoized_func(x):
        if x not in cache:
            cache[x] = f(x)
        return cache[x]
    memoized_func.__name__ = f.__name__
    return memoized_func

In [115]:
def call_counter(count_dictionary):
    def call_counter_decorator(func):
        def inner(*args, **kwargs):
            n = func.__name__
            #your code here
            if n in count_dictionary:
                count_dictionary[n] += 1
            else: 
                count_dictionary[n] = 1
            
            return func(*args)
        return inner
    return call_counter_decorator

### Q3. 

Use `cache` and `call_counter` as a decorator on `fib_recursive` and print the fibonacci numbers for 7,13,29, 33 as per last time. What order should these decorators be called to make sure `call_counter` gets the actual number of calls to `fib_recursive`?

In [163]:
#your code here
count_dict = {}

@cache
@call_counter(count_dict)
def fib_recursive(n):
    if n == 1:
        return 0
    if n == 2:
        return 1
    return fib_recursive(n-1) + fib_recursive(n-2)

In [147]:
for i in [7, 13, 29, 33]:
    print(fib_recursive(i))

8
144
317811
2178309


In [158]:
fib_recursive(29)

317811

In [159]:
count_dict

{'fib_recursive': 29}

In [164]:
fib_recursive(33)

2178309

In [165]:
count_dict

{'fib_recursive': 33}

### Notes to Self

In [ ]:
# TODO
count_dict = {}

#@call_counter(count_dict)
#@cache
def fib_recursive_nodec(n):
    if n == 1:
        return 0
    if n == 2:
        return 1
    return fib_recursive(n-1) + fib_recursive(n-2)

# the above decorator nesting is equiavlent to: 
# Goes INSIDE-OUT evaluating the definition
decorator_1 = cache
fib_recursive_1 = decorator_1(fib_recursive) # pass fib_recursive to cache and rebind to original name
# taking result of call_counter(count_dict), applying result to fib_recursive (see next cell)
decorator_2 = call_counter(count_dict)
fib_recursive_2 = decorator_2(fib_recursive_1) 
fib_recursive_nodec = fib_recursive_2
                                              
# Goes OUTSIDE-IN when calling function
fib_recursive(5)

In [ ]:
# Another way to see it with a missed cache: 

decorated_count_dict = {}
cache_miss_count_dict = {}

#@call_counter(decorated_count_dict) # counts all including cache calls
#@cache
#@call_counter(cache_miss_count_dict) # only counts underlying function calls to fib_recursive
def fib_recursive(n):
    if n == 1:
        return 0
    if n == 2:
        return 1
    return fib_recursive(n-1) + fib_recursive(n-2)

# Goes inside-out evaluating the definition
fib_recursive = cache(fib_recursive) # pass fib_recursive to cache and rebind to original name
# taking result of call_counter(count_dict), applying result to fib_recursive
fib_recursive = call_counter(count_dict)(fib_recursive) #see below to understand this syntax

# Goes outside-in when calling funciton
fib_recursive(5)

In [106]:
def f(x): 
    return (lambda y: y + x)

g = f(1)
print( g(2) ) # should be 1+2

print( (f(1))(3) )


def dumb(x):
    return lambda y: y + 1

3
4


In [107]:
f(2)
print(f(2)(3))

5


In [97]:
# showing how you can ignore things in the order of evaluating decorators

def ignore_and_return_3(f):
    return lambda x: 3

@ignore_and_return_3
def myfunc(x):
    return x * x

myfunc(17)

+++ 8
64
3


In [ ]:

def return_3_if_odd(f):
    def f_modified(x):
        if x % 2 == 0:
            return f(x)
        else:
            return 3
    return f_modified

def print_and_call(f):
    def f_with_printing(x):
        print('+++ ' + str(x))
        return f(x)
    return f_with_printing


@return_3_if_odd
@print_and_call
def square(x):
    return x * x

print(square(8))
print(square(9))

### Q4.

What is the time and space complexity of the the memoized Fibonacci? HINT: assume evaluation happens left to right on sub-expressions, so that `fib_recursive(n-1)` side of the tree is evaluated first, and thus the tree is evaluated depth first, from left to right. What kind of pruning happens in the tree?

*your answer here*

TODO

The time complexity is linear. The Space complexity is also linear. The entire right half of the tree is pruned off. 

### Q5.

Do you really need to store the entire array in the dynamic programming implementation? Isnt it enough to have only saved the previous two Fibonacci numbers? Implement such an algorithm in `fib_iterative2(n)`. What is its space and time complexity?

In [31]:
#your code here
def fib_iterative2(n):
    n_1 = 0
    n_2 = 1
    for i in range(2, n):
        temp = n_2
        n_2 = n_2 + n_1
        n_1 = temp
    return n_2

In [32]:
#your code here
for i in [1, 2, 7,13,29,33]:
    print(fib_iterative2(i))

1
1
8
144
317811
2178309


*your answer here*

Time complexity is O(n)

Space complexity is O(1) 

### Q6.

Write an algorithm for insertion sort. 

![](https://upload.wikimedia.org/wikipedia/commons/0/0f/Insertion-sort-example-300px.gif)

(from wikipedia)

The algorithm is also illustrated here: http://cs.armstrong.edu/liang/animation/web/InsertionSort.html and may be described thus

insertion sort is a method for sorting that starts with a single element (thus forming a trivially sorted list) and then incrementally inserts the remaining elements so that the list stays sorted.

Talk about the best, worst and average complexity of insertion sort. Use the same list A from the lecture
`A=[5,2,1,3,8,6,9]`

In [51]:
#your code here
def insertion_sort(alist):
    for i in range(1, len(alist)):
        j = i - 1
        element = alist[i]
        while element < alist[j] and j >= 0:
            alist[j+1] = alist[j]
            print(alist)
            j -= 1
        alist[j+1] = element
    return alist

In [83]:
#your code here
A=[9, 8, 7, 6, 5]
insertion_sort(A)

[5, 6, 7, 8, 9]

*your answer here*

Worst case insertion sort is $O(n^2)$ and the list is in reverse order. The outer loop and the inner loop  both run for $n \times n$ total loops. 

Best case is $O(n)$ if all the elements are already sorted then only the outer loop runs. 

Average case is $O(n^2)$. The outer loop always runs. The inner loop runs for any given comparison with a probability of 1/2 because a swap occurs when the left number is bigger than the number on the right, and there's equal probability that order could be either way. Therefore, the average time is n(n-1)/2. 